### 라이브러리 불러오기

In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.utils import shuffle
import time


device = torch.device('cuda')

# torch model 저장
def model_save(model, score,  path):
    os.makedirs('model', exist_ok=True)
    torch.save({
        'model': model.state_dict(),
        'score': score
    }, path)

### Seed 고정

In [2]:
# fix seed

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

### 이미지 불러오기

In [ ]:
train_png = sorted(glob('train/*.png'))
test_png = sorted(glob('test/*.png'))

In [3]:
train_y = pd.read_csv("train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

### 이미지 resize

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

In [ ]:
np.save('train_imgs_384', np.array(train_imgs))
np.save('test_imgs_384', np.array(test_imgs))

In [4]:
train_imgs = np.load('train_imgs_384.npy')
test_imgs = np.load('test_imgs_384.npy')

### Dataset 구성

In [5]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0.1)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
#class Custom_dataset(Dataset):
#    def __init__(self, img_paths, labels, mode='train'):
#        self.img_paths = img_paths
#        self.labels = labels
#        self.mode=mode
#    def __len__(self):
#        return len(self.img_paths)
#    def __getitem__(self, idx):
#        img = self.img_paths[idx]
#        if self.mode=='train':
#            augmentation = random.randint(0,3)
#            if augmentation==1:
#                img = img[::-1].copy()
#            elif augmentation==2:
#                img = img[:,::-1].copy()
#            elif augmentation==3:
#                img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
#            elif  augmentation==4:
#                img = cv2.blur(img,(10,10))
#            elif augmentation==4:
#                img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                       
#        img = transforms.ToTensor()(img)
#        img = transforms.Normalize([0.43309692, 0.40352088, 0.39421389], [0.18095295, 0.17348227, 0.16271882])(img)
#        if self.mode=='valid':
#            pass
#        if self.mode=='test':
#            pass
        
#        label = self.labels[idx]
#        return img, label
    
#class Network(nn.Module):
#    def __init__(self):
#        super(Network, self).__init__()
#        self.model = timm.create_model('efficientnet_lite0', pretrained=True, num_classes=88)
        
#    def forward(self, x):
#        x = self.model(x)
#        return x


In [ ]:
# KFold
#folds = []
#kf = KFold(n_splits=5, shuffle=True, random_state=42)
#for train_idx, valid_idx in kf.split(train_imgs):
#    folds.append((train_idx, valid_idx))
#fold=0
#train_idx, valid_idx = folds[fold]


#batch_size = 10
#epochs = 40


# Train
#train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
#train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)

# Validation 
#valid_dataset = Custom_dataset(np.array(train_imgs)[valid_idx], np.array(train_labels)[valid_idx], mode='valid')
#valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size, pin_memory=True)

# Test
#test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
#test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, pin_memory=True)

### 모델 학습

In [6]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [7]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 42,shuffle=True)
batch_size = 10
epochs = 25
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

  optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, 'best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------
-----------------SAVE:1 epoch----------------
epoch : 1/25    time : 386s/9258s
TRAIN    loss : 1.22769    f1 : 0.18214
Val    loss : 0.64904    f1 : 0.25837
-----------------SAVE:2 epoch----------------
epoch : 2/25    time : 377s/8663s
TRAIN    loss : 0.56710    f1 : 0.33233
Val    loss : 0.45906    f1 : 0.39706
-----------------SAVE:3 epoch----------------
epoch : 3/25    time : 379s/8334s
TRAIN    loss : 0.39972    f1 : 0.49300
Val    loss : 0.39391    f1 : 0.54554
-----------------SAVE:4 epoch----------------
epoch : 4/25    time : 377s/7914s
TRAIN    loss : 0.33171    f1 : 0.58484
Val    loss : 0.29687    f1 : 0.57541
-----------------SAVE:5 epoch----------------
epoch : 5/25    time : 378s/7554s
TRAIN    loss : 0.23723    f1 : 0.67541
Val    loss : 0.25391    f1 : 0.69890
-----------------SAVE:6 epoch----------------
epoch : 6/25    time : 378s/7188s
TRAIN    loss : 0.18800    f1 : 0.73965
Val    loss : 0.23232    f1 : 0.72038
-----------------S

-----------------SAVE:10 epoch----------------
epoch : 10/25    time : 385s/5770s
TRAIN    loss : 0.11206    f1 : 0.86597
Val    loss : 0.14726    f1 : 0.80150
epoch : 11/25    time : 383s/5361s
TRAIN    loss : 0.08823    f1 : 0.90040
Val    loss : 0.16129    f1 : 0.78338
-----------------SAVE:12 epoch----------------
epoch : 12/25    time : 385s/5000s
TRAIN    loss : 0.08550    f1 : 0.88364
Val    loss : 0.16304    f1 : 0.80535
-----------------SAVE:13 epoch----------------
epoch : 13/25    time : 385s/4618s
TRAIN    loss : 0.07854    f1 : 0.90756
Val    loss : 0.16468    f1 : 0.83819
epoch : 14/25    time : 386s/4241s
TRAIN    loss : 0.05790    f1 : 0.93415
Val    loss : 0.14371    f1 : 0.83235
epoch : 15/25    time : 384s/3840s
TRAIN    loss : 0.06835    f1 : 0.92529
Val    loss : 0.14929    f1 : 0.83039
-----------------SAVE:16 epoch----------------
epoch : 16/25    time : 385s/3467s
TRAIN    loss : 0.05747    f1 : 0.94482
Val    loss : 0.17647    f1 : 0.84834
epoch : 17/25    time

epoch : 20/25    time : 386s/1932s
TRAIN    loss : 0.03719    f1 : 0.96543
Val    loss : 0.17793    f1 : 0.78553
-----------------SAVE:21 epoch----------------
epoch : 21/25    time : 385s/1540s
TRAIN    loss : 0.04510    f1 : 0.95052
Val    loss : 0.19699    f1 : 0.83084
-----------------SAVE:22 epoch----------------
epoch : 22/25    time : 385s/1155s
TRAIN    loss : 0.05012    f1 : 0.94480
Val    loss : 0.14182    f1 : 0.83669
-----------------SAVE:23 epoch----------------
epoch : 23/25    time : 388s/776s
TRAIN    loss : 0.03785    f1 : 0.96078
Val    loss : 0.17263    f1 : 0.83793
epoch : 24/25    time : 387s/387s
TRAIN    loss : 0.03897    f1 : 0.96072
Val    loss : 0.33187    f1 : 0.80832
epoch : 25/25    time : 384s/0s
TRAIN    loss : 0.03057    f1 : 0.96602
Val    loss : 0.16530    f1 : 0.81753


In [ ]:
#model = Network().to(device)

#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
#criterion = nn.CrossEntropyLoss()
#scaler = torch.cuda.amp.GradScaler() 



#best=0
#for epoch in range(epochs):
#    start=time.time()
#    train_loss = 0
#    train_pred=[]
#    train_y=[]
#    model.train()
#    for batch in (train_loader):
#        optimizer.zero_grad()
#        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
#        y = torch.tensor(batch[1], dtype=torch.long, device=device)
#        with torch.cuda.amp.autocast():
#            pred = model(x)
#        loss = criterion(pred, y)


#        scaler.scale(loss).backward()
#        scaler.step(optimizer)
#        scaler.update()
        
#        train_loss += loss.item()/len(train_loader)
#        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
#        train_y += y.detach().cpu().numpy().tolist()
        
    
#    train_f1 = score_function(train_y, train_pred)

#    model.eval()
#    valid_loss = 0
#    valid_pred=[]
#    valid_y=[]
#    with torch.no_grad():
#        for batch in (valid_loader):
#            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
#            y = torch.tensor(batch[1], dtype=torch.long, device=device)
#            with torch.cuda.amp.autocast():
#                pred = model(x)
#            loss = criterion(pred, y)
#            valid_loss += loss.item()/len(valid_loader)
#            valid_pred += pred.argmax(1).detach().cpu().numpy().tolist()
#            valid_y += y.detach().cpu().numpy().tolist()
#        valid_f1 = score_function(valid_y, valid_pred)
#    if valid_f1>=best:
#        best=valid_f1
#        model_save(model, valid_f1, f'model/eff-b4.pth')
#    TIME = time.time() - start
#    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
#    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
#    print(f'VALID    loss : {valid_loss:.5f}    f1 : {valid_f1:.5f}    best : {best:.5f}')

### 추론

In [9]:
pred_ensemble = []
batch_size = 10

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load(('best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [10]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1]) + np.array(pred_ensemble[2]) + np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/5
f_pred = np.array(pred).argmax(1).tolist()

In [ ]:
#model.eval()
#f_pred = []

#with torch.no_grad():
#    for batch in (test_loader):
#        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
#        with torch.cuda.amp.autocast():
#            pred = model(x)
#        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [11]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [12]:
submission = pd.read_csv("sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [13]:
submission.to_csv("baseline26.csv", index = False)